In [2]:
import numpy as np
from scipy import sparse
from utils_classifier import create_bl_ratings_matrix, create_binary_bl_ratings_matrix

In [3]:
sparse_ratings_matrix = sparse.load_npz("../datasets/custom-ml/sparse_ratings_matrix_10000.npz")
sparse_ratings_matrix_5000 = sparse_ratings_matrix[:,:5000]
ratings_matrix_5000 = sparse_ratings_matrix_5000.toarray()

In [4]:
bl_ratings_matrix = create_bl_ratings_matrix(ratings_matrix_5000)
binary_bl_ratings_matrix = create_binary_bl_ratings_matrix(bl_ratings_matrix)

In [6]:
bl_ratings_matrix

array([[-1.,  1.,  1., ..., -1.,  1.,  1.],
       [ 1.,  1., -1., ..., -1.,  1.,  1.],
       [-1.,  1.,  1., ..., -1.,  1.,  1.],
       ...,
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.]])

In [17]:
mask = (bl_ratings_matrix == 1) | (bl_ratings_matrix == 0)
row_indices, column_indices,  = np.nonzero(mask)
data = bl_ratings_matrix[mask]

### Insert to mongo

In [14]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["cinematch_classification"]
collection_ratings_data = db["ratings_data"]

In [15]:
ratings_data_id = collection_ratings_data.insert_one({
    "n_users": 5000,
    "n_movies": 10000,
    "data":data.tolist()
})

In [19]:
collection_column_indices = db["ratings_column_indices"]
collection_row_indices = db["ratings_row_indices"]

collection_column_indices.insert_one({
    "data": column_indices.tolist(),
})

collection_row_indices.insert_one({
    "data": row_indices.tolist()
})

InsertOneResult(ObjectId('66484b9fd0b4be4b0c4e7406'), acknowledged=True)